# Sentiment Analysis

We have a set of customer reviews, and we need to classify them as “Positive” or “Negative”. We ask performers to read a review and decide which category it belongs to.

To get acquainted with Toloka tools for free, you can use the promo code **TOLOKAKIT1** on $20 on your [profile page](https://toloka.yandex.com/requester/profile?utm_source=github&utm_medium=site&utm_campaign=tolokakit) after registration.

Prepare environment and import all we'll need.

In [ ]:
!pip install toloka-kit==0.1.12
!pip install crowd-kit==0.0.5
!pip install ipyplot

import datetime
import sys
import time
import logging

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import toloka.client as toloka
import toloka.client.project.template_builder as tb
from crowdkit.aggregation import DawidSkene

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Сreate toloka-client instance. All api calls will go through it. More about OAuth token in our [Learn the basics example](https://github.com/Toloka/toloka-kit/tree/main/examples/0.getting_started/0.learn_the_basics) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/0.getting_started/0.learn_the_basics/learn_the_basics.ipynb)

In [ ]:
toloka_client = toloka.TolokaClient(input(), 'PRODUCTION') # Or switch to 'SANDBOX'
print(toloka_client.get_requester())

## Project creation

<b>Note</b>: The project name and description will be visible to the performers.

In [ ]:
project = toloka.Project(
    public_name='Classify customer reviews as positive or negative',
    public_description='Decide whether a review is positive or negative',
)

Create task interface. Read more about the Template Builder in the [Requester’s Guide](https://yandex.ru/support/toloka-tb/index.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

In [ ]:
review_view = tb.GroupViewV1(tb.TextViewV1(tb.InputData('review')))

radio_group_field = tb.ButtonRadioGroupFieldV1(
    tb.OutputData('sentiment'),
    [
        tb.GroupFieldOption('pos', 'Positive'),
        tb.GroupFieldOption('neg', 'Negative'),
    ],
    label='Is this review positive or negative?',
    validation=tb.RequiredConditionV1(),
)

task_width_plugin = tb.TolokaPluginV1(
    layout=tb.TolokaPluginV1.TolokaPluginLayout(
        kind='scroll',
        task_width=650,
    )
)

hot_keys_plugin = tb.HotkeysPluginV1(
    key_1=tb.SetActionV1(tb.OutputData('sentiment'), 'pos'),
    key_2=tb.SetActionV1(tb.OutputData('sentiment'), 'neg'),
)

project_interface = toloka.project.view_spec.TemplateBuilderViewSpec(
    view=tb.ListViewV1([review_view, radio_group_field]),
    plugins=[task_width_plugin, hot_keys_plugin],
)

<b>Note</b>: Specifications are a description of input data that will be used in a project and the output data that will be collected from the performers.

Read more about input and output data specifications in the [Requester’s Guide](https://yandex.ru/support/toloka-tb/operations/create-specs.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

In [ ]:
input_specification = {'review': toloka.project.field_spec.StringSpec()}
output_specification = {'sentiment': toloka.project.field_spec.StringSpec()}

Set task interface and data specifications to project.

In [ ]:
project.task_spec = toloka.project.task_spec.TaskSpec(
    input_spec=input_specification,
    output_spec=output_specification,
    view_spec=project_interface,
)

Write comprehensive instructions. Be sure to add examples for unobvious cases.

In [ ]:
project.public_instructions = """<p>In the task you will have to read customer reviews and define whether they are positive or negative</p>
<ul><li><b>Positive.</b> Choose this option if the review reflects a customer's first-hand <b>good</b> experience with the product recommending to purchase it. For your convenience, you can also use the short-cut by pressing "1"</li>
<li><b>Negative.</b> Choose this option if the review reflects a customer's first-hand <b>poor</b> experience with the product recommending <b>not</b> to purchase it. For your convenience, you can also use the short-cut by pressing "2"</li>
</ul>"""

Create a project.

In [ ]:
project = toloka_client.create_project(project)

## Create the main pool

A pool is a set of paid tasks grouped into task pages. These tasks are sent out for completion at the same time.

<b>Note</b>: All tasks within a pool have the same settings (price, quality control, etc.)

Specify the [pool parameters.](https://toloka.ai/docs/guide/concepts/pool_poolparams.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit)

Binary classification tasks are normally paid as basic tasks because these tasks do not take much time. Read more about [pricing principles](https://toloka.ai/knowledgebase/pricing?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in our Knowledge Base.

Choose `Languages.in_('EN')` as your first filter. This way, performers who speak English will be invited to complete this task.
Then choose `BROWSER` and `TOLOKA_APP` `ClientType` filters. These filters will make it possible for performers to complete your task on their computers or mobile devices.

In [ ]:
pool = toloka.Pool(
    project_id=project.id,
    # Give the pool any name you find suitable. You are the only one who will see it.
    private_name='Classify customer reviews as positive or negative',
    may_contain_adult_content=False,
    # Set the price per task suite.
    reward_per_assignment=0.01,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    # Overlap. This is the number of users who will complete the same task.
    # Set an overlap of 3 to get a more confident final label.
    defaults=toloka.Pool.Defaults(default_overlap_for_new_task_suites=3),
    # Specify the time given to complete a task suite (for example, 1200 seconds). To understand how much time it should take to
    # complete a task suite, try doing it yourself.
    assignment_max_duration_seconds=1200,
    # Filter performers who can access the task.
    filter=(toloka.filter.Languages.in_('EN')) & \
           (
                (toloka.filter.ClientType == 'TOLOKA_APP') |
                (toloka.filter.ClientType == 'BROWSER')
           )
)


Create a skill that will be used as a filter. A skill is a performer’s attribute which can be described by a number from 0 to 100 Skills usually reflect performer’s quality or can be set manually as a custom characteristic. Read more about skills [in the Requester’s Guide.](https://toloka.ai/docs/guide/concepts/nav.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit)

In [ ]:
sentiment_analysis_skill = next(toloka_client.get_skills(name='sentiment-analysis'), None)
if sentiment_analysis_skill:
    print('Sentiment analysis skill already exists')
else:
    sentiment_analysis_skill = toloka_client.create_skill(
        name='sentiment-analysis',
        # Choose whether the skill will be public. If your project will run for along time, public skills that reflect quality are
        # a good idea because they allow performers to track their own progress.
        hidden=True,
    )

Add a filter based on the skill you have just created. It will only allow access only to those performers who are either are new to the task (and have no quality attribute yet) or have decent quality calculated on the basis of golden-sets. See [Set up quality control](set-up-quality-control.) section. for details on how to set up quality calculation.

You will be able to use the same skill as a filter in other pools, keeping a project-based quality history for your performers.

In [ ]:
skill_filter = (toloka.filter.Skill(sentiment_analysis_skill.id) == None) | \
               (toloka.filter.Skill(sentiment_analysis_skill.id) >= 90)

Add filter to pool.

In [ ]:
pool.set_filter(pool.filter & skill_filter)

## Set up Quality control.

Ban performers who give incorrect responses to control tasks.

Since tasks such as these have an answer that can be used as a ground truth, we can use standard quality control rules such as golden sets.

Read more about [quality control principles](https://toloka.ai/knowledgebase/quality-control?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in our Knowledge Base or [check out control tasks settings](https://toloka.ai/docs/guide/concepts/goldenset.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in the Requester’s Guide.

Set up a rule for Captcha. It will be shown at low frequency and those performers who are not accurate will be suspended in the pool for a day.

Captcha is a good tool to check performers’ attention. Read more about different Quality Control rules in [Toloka Knowledge Base.](https://toloka.ai/knowledgebase/quality-control?utm_source=github&utm_medium=site&utm_campaign=tolokakit)


In [ ]:
# Turns on captchas
pool.set_captcha_frequency('MEDIUM')

pool.quality_control.add_action(
    collector=toloka.collectors.Captcha(history_size=10),
    conditions=[
        toloka.conditions.StoredResultsCount >= 4,
        toloka.conditions.SuccessRate < 75,
    ],
    action=toloka.actions.RestrictionV2(
        scope='PROJECT',
        duration=1,
        duration_unit='DAYS',
        private_comment='captcha'
    )
)

Set up the [Fast responses rule](https://toloka.ai/docs/guide/concepts/quick-answers.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit). It allows to ban performers who submit tasks at a suspicious speed.

In [ ]:
pool.quality_control.add_action(
    collector=toloka.collectors.AssignmentSubmitTime(fast_submit_threshold_seconds=20),
    conditions=[
        toloka.conditions.TotalSubmittedCount > 4,
        toloka.conditions.FastSubmittedCount > 2,
    ],
    action=toloka.actions.RestrictionV2(
        scope='PROJECT',
        duration=1,
        duration_unit='DAYS',
        private_comment='fast responses'
    )
)

Set up individual quality calculation based on control tasks. Each performer’s results will be recorded in the skill you have created earlier.

Read more about configurating this rule in our [Requester’s Guide](https://toloka.ai/docs/guide/concepts/goldenset.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

In [ ]:
pool.quality_control.add_action(
    collector=toloka.collectors.GoldenSet(history_size=10),
    conditions=[
        toloka.conditions.TotalAnswersCount > 4,
    ],
    action=toloka.actions.SetSkillFromOutputField(
        skill_id=sentiment_analysis_skill.id,
        from_field='correct_answers_rate'
    )
)

Set up the [Submitted responses](https://toloka.ai/docs/guide/concepts/submitted-answers.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) rule. This is used to get more variety in answers so that the answers won’t be biased toward only a few productive performers.

In [ ]:
pool.quality_control.add_action(
    collector=toloka.collectors.AnswerCount(),
    conditions=[
        toloka.conditions.AssignmentsAcceptedCount >= 30,
    ],
    action=toloka.actions.RestrictionV2(
        scope='PROJECT',
        duration=1,
        duration_unit='DAYS',
        private_comment='too many responses'
    )
)

Set Smart mixing option in pool settings and specify the number of tasks of each type per page. We recommend to put as much tasks in one page as a performer can complete in 1 to 5 minutes. This volume does not let performers get tired and protects them from significant data losses in case of a technical issue. To learn more how to group tasks in suites, read the Requester’s Guide.

In [ ]:
pool.set_mixer_config(real_tasks_count=9, golden_tasks_count=1, training_tasks_count=0)

Create pool.

In [ ]:
pool = toloka_client.create_pool(pool)

## Prepare and upload tasks

We will use [Grammar and Online Product Reviews](https://data.world/datafiniti/grammar-and-online-product-reviews) dataset under Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International license


[![CC BY-NC-SA 4.0](https://img.shields.io/badge/License-CC%20BY--NC--SA%204.0-lightgrey.svg)](https://creativecommons.org/licenses/by-nc-sa/4.0/).

In [ ]:
!curl https://tlk.s3.yandex.net/ext_dataset/datafiniti_grammar_and_online_product_reviews.csv --output dataset.csv

dataset = pd.read_csv('dataset.csv')
dataset = dataset.sample(frac=1)[['reviews.text', 'reviews.doRecommend']].dropna().reset_index(drop=True)
dataset = dataset.replace({'reviews.doRecommend': {True: 'pos', False: 'neg'}})

print(f'\nDataset contains {len(dataset)} rows')

dataset.sample(5)

Control tasks are tasks that already contain the correct response. They are used for checking the quality of responses from performers. The performer's response is compared to the response you provided. If they match, it means the performer answered correctly.

<b>Tip.</b> Make sure to include different variations of correct responses in equal amounts.

In [ ]:
task_dataset_size = 200
golden_dataset_size = int(task_dataset_size * 0.1)

# We will balance classes both in golden and task datasets

positive_tasks = dataset[dataset['reviews.doRecommend'] == 'pos']
negative_tasks = dataset[dataset['reviews.doRecommend'] == 'neg']

pos_task_dataset, pos_golden_dataset, _ = np.split(
    positive_tasks,
    [task_dataset_size // 2, task_dataset_size // 2 + golden_dataset_size // 2]
)
neg_task_dataset, neg_golden_dataset, _ = np.split(
    negative_tasks,
    [task_dataset_size // 2, task_dataset_size // 2 + golden_dataset_size // 2]
)

task_dataset = pd.concat([pos_task_dataset, neg_task_dataset]).sample(frac=1)
golden_dataset = pd.concat([pos_golden_dataset, neg_golden_dataset]).sample(frac=1)

golden_tasks = [
    toloka.Task(
        pool_id=pool.id,
        input_values={'review': row['reviews.text']},
        known_solutions = [
            toloka.task.BaseTask.KnownSolution(
                output_values={'sentiment': row['reviews.doRecommend']}
            )
        ],
        infinite_overlap=True,
    )
    for _, row in golden_dataset.iterrows()
]

tasks = [
    toloka.Task(
        pool_id=pool.id,
        input_values={'review': review},
    )
    for review in task_dataset['reviews.text']
]

Upload tasks

In [ ]:
created_tasks = toloka_client.create_tasks(golden_tasks + tasks, allow_defaults=True)
print(len(created_tasks.items))

You can go to the pool preview page and in web-interface you can see something like this:
<table  align="center">
  <tr><td>
    <img src="./img/pool_preview.png"
         alt="Pool interface"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> What the pool interface might look like.
  </td></tr>
</table>

Start the pool.

<b>Note</b>: Remember that the tasks will be completed by actual Tolokers. Double check that everything is correct with your project configuration.

In [ ]:
pool = toloka_client.open_pool(pool.id)
print(pool.status)

## Receiving responses

Wait until the pool is completed.

In [ ]:
pool_id = pool.id

def wait_pool_for_close(pool_id, minutes_to_wait=1):
    sleep_time = 60 * minutes_to_wait
    pool = toloka_client.get_pool(pool_id)
    while not pool.is_closed():
        op = toloka_client.get_analytics([toloka.analytics_request.CompletionPercentagePoolAnalytics(subject_id=pool.id)])
        op = toloka_client.wait_operation(op)
        percentage = op.details['value'][0]['result']['value']
        print(
            f'   {datetime.datetime.now().strftime("%H:%M:%S")}\t'
            f'Pool {pool.id} - {percentage}%'
        )
        time.sleep(sleep_time)
        pool = toloka_client.get_pool(pool.id)
    print('Pool was closed.')

wait_pool_for_close(pool_id)

Get responses.

In [ ]:
answers_df = toloka_client.get_assignments_df(pool_id)

# Drop golden tasks
answers_df = answers_df[answers_df['GOLDEN:sentiment'].isna()]

# Prepare DataFrame for aggregation
answers_df = answers_df.rename(columns={
    'INPUT:review': 'task',
    'OUTPUT:sentiment': 'label',
    'ASSIGNMENT:worker_id': 'performer'
})

print(f'answers count: {len(answers_df)}')

Run aggregation using the [Dawid-Skene](https://toloka.ai/docs/guide/concepts/result-aggregation.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit#aggr__dawid-skene) model.

We use this aggregation model because our questions are of the same difficulty, and we don't have many control tasks.

Read more about the Dawid-Skene model in the Requester’s Guide or get at an overview of different aggregation models in our Knowledge Base.


In [ ]:
# Run aggregation
predicted_answers = DawidSkene(n_iter=20).fit_predict(answers_df)

Look at the results.

In [22]:
pd.DataFrame({'review': predicted_answers.index, 'sentiment': predicted_answers.values}).sample(10)

,review,sentiment
171,"Oddly enough I wrote Rubbermaid on line today to mention how my mop unexpectedly snapped at where the metal handle meets the plastic base. I seldom used the fluid bottle, hence why my sprayer handle never broke! I then browsed and found this forum where over 80 reviews that I read quickly all had little positive to say about this product regarding construction or reliability. In concept, it is perfect for ceramic floors, but another one goes to the landfill, as there is no repairing it based on a poor design. What a frustration, based on retail price, and a Canadian 13 sales tax on top.",neg
165,"I would wait until this movie is less than $5 to buy it, or borrow it from a friend",neg
26,Great movie. Disney never disappoints when they do a classic,pos
7,Got it because it was 3DOther wise would have got the bootleg for $2,pos
89,Great!!!! I love it best ever I had an going to get it every time,pos
38,"I got this to review for free, but I've honestly tried better conditioners out there. I like the shampoo and using this together just completes it. I love the smell of it, but I wish it was more moisturizing for my ends! This review was collected as part of a promotion.",pos
143,5 dics for 15 can't beat that even if i don't like some of them.,neg
46,It's light and refreshing-takes away my craving for sweets.,pos
16,great cleaning product and so easy to use they make your house clean and disenfectecd This review was collected as part of a promotion.,neg
137,"I have short and medium coiled curly hair, and it takes away any residue because I can see the color from some product I was using of the ingredients! Even now my hair still feels softer, and nicer. I love how it feels. This shampoo is the only one that make my hair feel so different, and will be repurchased!",neg
